# load libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
import json
import bs4 as bs
import urllib.request
import pickle
import requests

In [6]:
import warnings
warnings.filterwarnings('ignore')

# Load .pkl files

In [7]:
# load the nlp model and tfidf vectorizer from pkl 
filename = 'trained_sentiment_model.pkl'
clf = pickle.load(open(filename, 'rb'))
vectorizer = pickle.load(open('transform.pkl','rb'))

# load movie dataset

In [23]:
## laod movie dataset

df = pd.read_csv('final_final_data.csv')
df.head()

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...


# get imdb id for movies

In [28]:
import requests

def get_imdb_id(movie_title, api_key):
    base_url = "http://www.omdbapi.com/"
    params = {
        "apikey": api_key,
        "t": movie_title
    }

    try:
        response = requests.get(base_url, params=params)
        data = response.json()

        if data["Response"] == "True":
            print(f'{movie_title}: imdb_id:{data["imdbID"]}')
            return data["imdbID"]
        else:
            print("Movie not found or API error:", data["Error"])
            return None

    except requests.RequestException as e:
        print("Error occurred during API call:", e)
        return None



In [33]:
# Replace 'YOUR_API_KEY' with your actual OMDb API key
api_key =  '16ed114'   ###"YOUR_API_KEY"

# Example usage
movie_title = "Iron Man"
imdb_id = get_imdb_id(movie_title,api_key )


Iron Man: imdb_id:tt0371746


In [37]:
df_500 = df.iloc[:500]

In [38]:
df_500['imdb_id']= df_500['movie_title'].map(lambda x:get_imdb_id(x,api_key))

avatar: imdb_id:tt0499549
pirates of the caribbean: at world's end: imdb_id:tt0449088
spectre: imdb_id:tt2379713
the dark knight rises: imdb_id:tt1345836
star wars: episode vii - the force awakens            : imdb_id:tt2488496
john carter: imdb_id:tt0401729
tangled: imdb_id:tt0398286
avengers: age of ultron: imdb_id:tt2395427
harry potter and the half-blood prince: imdb_id:tt0417741
batman v superman: dawn of justice: imdb_id:tt2975590
superman returns: imdb_id:tt0348150
quantum of solace: imdb_id:tt0830515
pirates of the caribbean: dead man's chest: imdb_id:tt0383574
the lone ranger: imdb_id:tt1210819
man of steel: imdb_id:tt0770828
the chronicles of narnia: prince caspian: imdb_id:tt0499448
pirates of the caribbean: on stranger tides: imdb_id:tt1298650
men in black 3: imdb_id:tt1409024
the hobbit: the battle of the five armies: imdb_id:tt2310332
the amazing spider-man: imdb_id:tt0948470
robin hood: imdb_id:tt0955308
the hobbit: the desolation of smaug: imdb_id:tt1170358
the golden c

# sentiment analysis

In [60]:
pd.set_option('display.max_colwidth', None)

In [61]:
def get_sentiment(title):
    # get imdb id
    imdb_id = df_500.loc[df_500['movie_title']==title]['imdb_id'].iloc[0].strip() 
    # web scraping to get user reviews from IMDB site
    sauce = urllib.request.urlopen('https://www.imdb.com/title/{}/reviews?ref_=tt_ov_rt'.format(imdb_id)).read()
    soup = bs.BeautifulSoup(sauce,'lxml')
    soup_result = soup.find_all("div",{"class":"text show-more__control"})
    reviews_list = [] # list of reviews
    reviews_status = [] # list of comments (good or bad)
    for reviews in soup_result:
        if reviews.string:
            reviews_list.append(reviews.string)
            # passing the review to our model
            movie_review_list = np.array([reviews.string])
            movie_vector = vectorizer.transform(movie_review_list)   ### sentiment analysis
            pred = clf.predict(movie_vector)
            reviews_status.append('Good' if pred else 'Bad')
    return pd.DataFrame({'Reviews':reviews_list, 'sentiment': reviews_status})

In [66]:
get_sentiment('the dark knight rises')

,Reviews,sentiment
0,"Christopher Nolan's epic trilogy concludes in glorious fashion and gives us a thought provoking and suitably satisfying conclusion to an epic saga. It's emotional, intense and has a great villain in Tom Hardy.",Good
1,"I think as a film stand point, the dark knight is a better movie that is better acted and tense the whole way through. The dark knight rises to me is a more fun and action filled movie that you can sit back a bit more. The acting is still amazing and combined with the atmosphere/visuals of the movie creates a the sense of realism that the movie has. That realism is where the intensity stems from throughout the movie, the audience believing that something like this could actually happen makes it that much scarier. Bane is also an excellent villain, in the dark knight the joker challenges Batman mentally but Bane physically challenges him which is a completely new challenge for Batman. The plot is incredible, it it starts fast and instantly brings you into the action. It then takes a step back to develop the characters further and build up the challenge Batman has to face. Once one of the many challenges was overcome by Batman in launches the viewer straight back into action for one epic final battle that lasts almost an hour......and it is GLORIOUS! Not one second was I bored, it is truly incredible how each movie in the trilogy is this good, it shouldn't be possible. I've not even mentioned the incredible soundtrack to compliment the astounding yet grounded action. I can't compliment this movie enough. If you haven't someone seen this trilogy yet....watch it, you won't be disappointed.",Good
2,"The Dark Knight Rises was a perfect sequel to a masterpiece. Christopher Nolan took a different direction and focused more on Bruce Wayne/Batman. This movie is filled with lots of emotions. It has such great characters and of course the dialogs is so good. The performances are incredible. It was my most anticipated movie of the century. The more I watch it, The more epic it gets. Personally, I think The Dark Knight Trilogy is the greatest Trilogy of all time and this movie gives the greatest ending of all time in the movie history. I remember seeing it thrice and the more I watched It the more respect it earns. Thank you Christopher Nolan, You Sir Are A Master.",Good
3,"Possibly my personal favourite of the Nolan Batman films, and I do like Batman Begins and The Dark Knight very much. Is it perfect? For me, not quite, there are so many plot strand in a long running time that the film occasionally(and there is a big emphasis on that) sags in momentum and some of the strands in the ending could have done with more development. It is a brilliant film all the same, one of the highlights so far of the year. As to expect from Christopher Nolan, it is a visually audacious film, with gorgeous cinematography, a beautifully realised Gotham City and great special effects. Hans Zimmer's score is not one of his very best for me, but there is still an epic and haunting quality that is just perfect for this film. The dialogue provokes thought and makes an emotional impact, and a vast majority of the time The Dark Knight Rises is ambitious and gripping. I don't consider Christopher Nolan one of my favourites, this said he is a hugely impressive director who shows great ambition and a willingness to make his subject matters interesting. He does a really wonderful job here, as the set/action pieces including the plane hijack and Bane's assault on Gotham are enough to make the jaw drop. The performances are just terrific from all involved, Matthew Modine fares weakest but he is far from bad. Christian Bale is still a charismatic presence, Matthew Caine appropriately loyal and Gary Oldman and Morgan Freeman subtly understated. Joseph Gordon Levitt captures his character's idealism wonderfully. Tom Hardy is in beefed-up terrifying mode as Bane, a worthy successor to Heath Ledger's iconic Joker. Anne Hathaway was a curiou